In [1]:
import pandas

In [2]:
import mlflow
import ipywidgets as widgets
from ipywidgets import interact
import os
import glob
import torch
from pprint import pprint
from easydict import EasyDict

import numpy as np
import torch
import yaml

In [3]:
MLFLOW_TRACKING_URI = "/home/home01/scrb/01_repos/CardiacMotion/mlruns/"
EXPERIMENT_NAME = "Synthetic data"
EXPERIMENT_NAME = "test"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [4]:
MLFLOW_URI = mlflow.tracking.get_tracking_uri()
EXPERIMENT_ID = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

PREFIX = f"{MLFLOW_URI}/{EXPERIMENT_ID}"

In [5]:
meta_yamls = [f"{PREFIX}/{run}/meta.yaml" for run in os.listdir(PREFIX) if os.path.exists(f"{PREFIX}/{run}/meta.yaml")]

count = 0
for meta_yaml_path in meta_yamls:
    meta_yaml = yaml.safe_load(open(meta_yaml_path))    
    if meta_yaml['experiment_id'] != EXPERIMENT_ID:
        meta_yaml['experiment_id'] = EXPERIMENT_ID
        count += 1
        yaml.dump(meta_yaml, open(meta_yaml_path, "wt"))
        
if count != 0:
    print(f"{count} runs's experiments were fixed to match the experiment of the parent folder")

In [6]:
df = mlflow.search_runs(experiment_ids=[EXPERIMENT_ID])
df = df[df["metrics.test_rec_ratio_to_time_mean"] < 0.9]
df

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.val_rec_ratio_to_pop_mean,metrics.training_recon_loss_s,metrics.val_recon_loss_s,metrics.training_recon_loss_c,...,params.platform,params.min_delta,params.dataset_complexity,params.dataset_complexity_c,tags.mlflow.log-model.history,tags.mlflow.source.type,tags.mlflow.source.git.commit,tags.Mode,tags.mlflow.user,tags.mlflow.source.name
1,6a4d73fb59f24d97b37764afdedd4185,2,FINISHED,/root/Rodrigo_repos/CardiacMotion/mlruns/2/6a4...,2023-03-31 11:49:30.571000+00:00,2023-04-01 06:02:43.379000+00:00,0.120440,1.647883,2.157799,1.445723,...,b3910956944c,-0.0,36,9,"[{""run_id"": ""6a4d73fb59f24d97b37764afdedd4185""...",LOCAL,7503005beeed87c75dfdd7289540bfd8074f1221,testing,root,main_autoencoder_cardiac.py
21,063d59b768a846be94c27e2fcb0f8cfe,2,FINISHED,/root/Rodrigo_repos/CardiacMotion/mlruns/2/063...,2023-03-23 10:00:30.673000+00:00,2023-03-24 02:34:12.351000+00:00,0.144234,0.002296,0.002706,0.002451,...,332c5cd2476e,-0.0,36,9,"[{""run_id"": ""063d59b768a846be94c27e2fcb0f8cfe""...",LOCAL,7503005beeed87c75dfdd7289540bfd8074f1221,testing,root,main_autoencoder.py


___

In [7]:
### Select runs with $T=1$

# row_index = (df["params.dataset_n_timeframes"] == "1")
# 
# # Select runs with performance better than random
# row_index &= (df["metrics.test_rec_ratio_to_pop_mean"] < 1)

In [8]:
# columns
normalized_metrics = df.columns[df.columns.str.startswith("metrics.test_") & df.columns.str.contains("ratio")]

dataset_params = df.columns[df.columns.str.startswith("params.dataset_")].to_list()

params = df.columns[df.columns.str.startswith("params.")].to_list()

arch_params = [
    'params.n_channels_enc', 
    'params.n_channels_dec_c', 
    'params.n_channels_dec_s', 
    'params.latent_dim_c', 
    'params.latent_dim_s',
    'params.z_aggr_function',
    'params.reduction_factors',
]

# To not display columns that have the same value for all rows
# https://stackoverflow.com/questions/57365283/how-to-show-columns-that-have-different-values-in-rows
def diff_cols(df):
    my_cols = []
    for col in df.columns:
        if df[col].nunique(dropna=False) > 1:
            my_cols.append(col)
    return df[my_cols].copy()

In [9]:
columns = ["run_id"] + params + normalized_metrics.tolist()

diff_cols(
    df[columns].reset_index(drop=True)
)

,run_id,params.lr,params.latent_dim_s,params.reduction_factors,params.dataset_n_timeframes,params.latent_dim_c,params.w_s,params.platform,metrics.test_rec_ratio_to_time_mean,metrics.test_rec_ratio_to_pop_mean_c,metrics.test_rec_ratio_to_pop_mean
0,6a4d73fb59f24d97b37764afdedd4185,0.0001,16,"[3, 3, 3, 3]",50,8,0.1,b3910956944c,0.854538,0.002284,0.113931
1,063d59b768a846be94c27e2fcb0f8cfe,3e-05,36,"[2, 2, 2, 2]",20,9,1.0,332c5cd2476e,0.629545,0.007206,0.106766


In [10]:
good_runs = df.run_id.values
run_w = widgets.Select(options=good_runs)

### Get run IDs based on metrics 

In [11]:
# run_w = widgets.Select(options=["6a4d73fb59f24d97b37764afdedd4185"])
run_w

Select(options=('6a4d73fb59f24d97b37764afdedd4185', '063d59b768a846be94c27e2fcb0f8cfe'), value='6a4d73fb59f24d…

### Load weights

In [12]:
# ckpt_dir = f"/root/Rodrigo_repos/CardiacMotion/2/{run_w.value}/checkpoints"
# ckpt_path = f"{ckpt_dir}/{os.listdir(ckpt_dir)[0]}"
ckpt_dir = f"{os.environ['HOME']}/01_repos/CardiacMotion/mlruns/2/{run_w.value}/artifacts/restored_model_checkpoint"
ckpt_path = f"{ckpt_dir}/{os.listdir(ckpt_dir)[0]}"

In [13]:
# ckpt_path = f"/root/Rodrigo_repos/CardiacMotion/2/{run_w.value}/checkpoints/epoch=334-step=38524.ckpt"
model_weights = torch.load(ckpt_path, map_location=torch.device('cpu'))["state_dict"]
print(f"Loaded weights from checkpoint:\n {ckpt_path}")
model_weights = EasyDict(model_weights)

Loaded weights from checkpoint:
 /home/home01/scrb/01_repos/CardiacMotion/mlruns/2/6a4d73fb59f24d97b37764afdedd4185/artifacts/restored_model_checkpoint/epoch=334-step=38524.ckpt


### Initialize weights

In [14]:
import sys
import pickle as pkl
import os

os.environ["HOME"] = "/home/home01/scrb"
os.environ['CARDIAC_MOTION_REPO'] = os.environ["HOME"] + "/01_repos/CardiacMotion"
os.chdir(os.environ['CARDIAC_MOTION_REPO'])

sys.path.append(os.environ['CARDIAC_MOTION_REPO'])

from main_autoencoder_cardiac import *
from config.load_config import load_yaml_config

#### Cardiac dataset

In [15]:
from main_autoencoder_cardiac import *

In [16]:
POLYNOMIAL_DEGREE = 6
DOWNSAMPLING = 2

config = load_yaml_config("config_folded_c_and_s.yaml")
config.network_architecture.convolution.parameters.polynomial_degree = [POLYNOMIAL_DEGREE] * 4
config.network_architecture.pooling.parameters.downsampling_factors = [DOWNSAMPLING] * 4

In [17]:
faces = EasyDict(
        pkl.load(open("utils/VTKHelpers/data/faces_and_downsampling_mtx_frac_0.1_LV.pkl", "rb"))
).new_faces

template = EasyDict({
  "v": np.load(f"{os.environ['CARDIAC_MOTION_REPO']}/data/LV_shape_mean_across_timepoints.npy"),
  "f": faces
})


cardiac_dataset = CardiacMeshPopulationDataset(
    root_path=f"{os.environ['HOME']}/01_repos/CardiacMotion/data/cardio/Results", 
    procrustes_transforms="utils/VTKHelpers/data/procrustes_transforms_35k.pkl",
    faces=faces,
    template_mesh=template,        
)

In [19]:
mesh_dm = CardiacMeshPopulationDM(cardiac_dataset, batch_size=32)

# datamodule = get_datamodule(config.dataset, batch_size=config.batch_size)

config.network_architecture.latent_dim_c = 8 
config.network_architecture.latent_dim_s = 16

mesh_dm.setup()
x = EasyDict(next(iter(mesh_dm.train_dataloader())))

mesh_template = mesh_dm.dataset.template_mesh
coma_args = get_coma_args(config)
coma_matrices = get_coma_matrices(config, mesh_template)
coma_args.update(coma_matrices)

In [20]:
from models.Model3D import Encoder3DMesh, Decoder3DMesh
from models.Model4D import DECODER_C_ARGS, DECODER_S_ARGS, ENCODER_ARGS
from models.Model4D import DecoderStyle, DecoderContent, DecoderTemporalSequence 
from models.Model4D import EncoderTemporalSequence, AutoencoderTemporalSequence
from lightning.ComaLightningModule import CoMA_Lightning

from models.lightning.EncoderLightningModule import TemporalEncoderLightning
from models.TemporalAggregators import TemporalAggregator, FCN_Aggregator

enc_config = EasyDict({k: v for k, v in coma_args.items() if k in ENCODER_ARGS})
encoder = Encoder3DMesh(**enc_config)

enc_config.latent_dim = config.network_architecture.latent_dim_c + config.network_architecture.latent_dim_s 

h = encoder.forward_conv_stack(x.s_t, preserve_graph_structure=False)

NT = 50 # config.dataset.parameters.T
    
z_aggr = FCN_Aggregator(
    features_in = NT*h.shape[-1],
    features_out= enc_config.latent_dim
)

t_encoder = EncoderTemporalSequence(
    encoder3d = encoder,
    z_aggr_function=z_aggr
)

decoder_config_c = EasyDict({ k:v for k,v in coma_args.items() if k in DECODER_C_ARGS })
decoder_config_s = EasyDict({ k:v for k,v in coma_args.items() if k in DECODER_S_ARGS })    
decoder_content = DecoderContent(decoder_config_c)
decoder_style = DecoderStyle(decoder_config_s, phase_embedding_method="exp_v1")
t_decoder = DecoderTemporalSequence(decoder_content, decoder_style)
    
t_ae = AutoencoderTemporalSequence(
    encoder=t_encoder,
    decoder=t_decoder
)

OSError: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /nobackup/scrb/.conda/envs/cardio/lib/python3.9/site-packages/torch_spline_conv/_basis_cuda.so)

In [ ]:
model_weights = EasyDict({k.replace("model.", ""): v for k, v in model_weights.items()})

In [ ]:
t_ae.load_state_dict(model_weights)

In [ ]:
lit_module = CoMA_Lightning(
    model=t_ae, 
    loss_params=config.loss, 
    optimizer_params=config.optimizer,
    additional_params=config,
    mesh_template=mesh_template
)

### Load input meshes

In [ ]:
x = next(iter(mesh_dm.val_dataloader()))

In [ ]:
output = t_ae(x["s_t"])

### Generate animation

In [ ]:
s_t, s_hat_t = x["s_t"], output[2]

In [ ]:
# subj_idx_w = widgets.IntSlider(min=1, max=len(cardiac_dataset))

def generate_gif(mesh4D, faces, filename, camera_position='xy', show_edges=False, **kwargs):
        
        '''
        Produces a gif file representing the motion of the input mesh.
        
        params:
          ::mesh4D:: a sequence of Trimesh mesh objects.
          ::faces:: array of F x 3 containing the indices of the mesh's triangular faces.
          ::filename:: the name of the output gif file.
          ::camera_position:: camera position for pyvista plotter (check relevant docs)
          
        return:
          None, only produces the gif file.
        '''

        import pyvista as pv
        
        connectivity = np.c_[np.ones(faces.shape[0]) * 3, faces].astype(int)
                
        pv.set_plot_theme("document")
        os.makedirs(os.path.dirname("./"+filename) , exist_ok=True)
        
        # plotter = pv.Plotter(shape=(1, len(camera_positions)), notebook=False, off_screen=True)
        pv.start_xvfb()
        plotter = pv.Plotter(notebook=False, off_screen=True)
            
        # Open a gif
        plotter.open_gif(filename)

        try:
            # if mesh3D is torch.Tensor, this your should run OK
            mesh4D = mesh4D.cpu().numpy()[0].astype("float32")
        except AttributeError:
            pass

        kk = pv.PolyData(mesh4D[0], connectivity)
        # plotter.add_mesh(kk, smooth_shading=True, opacity=0.5 )#, show_edges=True)
        plotter.add_mesh(kk, show_edges=show_edges, opacity=0.5, color="red") 
        
        for t, _ in tqdm(enumerate(mesh4D)):
            # print(t)
            kk = pv.PolyData(mesh4D[t], connectivity)
            plotter.camera_position = camera_position
            plotter.update_coordinates(kk.points, render=False)
            plotter.render()             
            plotter.write_frame()
        
        plotter.close()
        
        return filename

In [ ]:
from tqdm.notebook import trange, tqdm

In [ ]:
import numpy as np
from PIL import Image
import imageio

def merge_gifs_horizontally(gif_file1, gif_file2, output_file):
    # Create reader object for the gif
    gif1 = imageio.get_reader(gif_file1)
    gif2 = imageio.get_reader(gif_file2)

    # Create writer object
    new_gif = imageio.get_writer(output_file)

    for frame_number in range(gif1.get_length()):
        img1 = gif1.get_next_data()
        img2 = gif2.get_next_data()
        # here is the magic
        new_image = np.hstack((img1, img2))
        new_gif.append_data(new_image)

    gif1.close()
    gif2.close()
    new_gif.close()

### Generate gif's

In [ ]:
subj_ids = list(range(64))

In [ ]:
for subj_id in subj_ids:
        
    mesh4D = s_hat_t.detach().cpu().numpy().astype("float32")[subj_id]    
    for camera in ["xz", "xy", "yz"]:    
        gifpath = generate_gif(
            mesh4D,
            faces, 
            camera_position=camera,
            filename=f"id{subj_id}_reconstruction_{camera}.gif"
        )
        
       #b64 = base64.b64encode(
       #  open(gifpath,'rb').read()
       #).decode('ascii')
        
                
    mesh4D = s_t.detach().cpu().numpy().astype("float32")[subj_id]        
    for camera in ["xz", "xy", "yz"]:    
        gifpath = generate_gif(
            mesh4D,
            faces, 
            camera_position=camera,
            filename=f"id{subj_id}_original_{camera}.gif", 
        )
    
        #b64 = base64.b64encode(
        #    open(gifpath,'rb').read()
        #).decode('ascii')
    
    for camera in ["xz", "xy", "yz"]: 
        
        merge_gifs_horizontally(
            f"id{subj_id}_original_{camera}.gif", 
            f"id{subj_id}_reconstruction_{camera}.gif", 
            f"id{subj_id}_{camera}.gif"
        )
        
        os.remove(f"id{subj_id}_original_{camera}.gif")
        os.remove(f"id{subj_id}_reconstruction_{camera}.gif")

In [ ]:
@interact
def show_gif(subj_id=widgets.IntSlider(min=0,max=63)):
    
    from IPython.display import HTML
    import base64
   
    
    
    display(HTML(f'<img src="data:image/gif;base64,{b64}" />'))

In [ ]:
subj_id = "15"
